In [1]:
import copy
import numpy as np
import json
from types import SimpleNamespace
from operator import itemgetter
from sklearn.preprocessing import StandardScaler


def data_from_json():
    with open('all_artcles.json', encoding="utf8") as file: 
        data = json.load(file, object_hook=lambda d: SimpleNamespace(**d))
        
    list_X_Y = []
    list_BET = []
    list_info = []
    
    for article in data:
            for graph in article.Graphs:
                for izotherm in graph.Isotherms:            
                    if( izotherm.SampleFeatures.Total_surface_area != "" and (izotherm.FileName != "Data2" and izotherm.FileName != "Data21" and izotherm.FileName != "Data73") and izotherm.SampleName != "UAK2-800d-without_washing" and izotherm.SampleName != "SAC600_3_1" ):                                      
                        if( len(izotherm.XAxisData) == len(izotherm.YAxisData)):
                            temp_list_x = []
                            temp_list_y = []
                            for x in izotherm.XAxisData:
                                if(x<0 or x==0):
                                    x = abs(x)
                                temp_list_x.append(x)
                            for y in izotherm.YAxisData:
                                if(y<0 or x==0):
                                    y = abs(y)
                                temp_list_y.append(y)
                            temp_list_x_y =[]
                            for i in range(len(temp_list_x)):
                                small_list = [temp_list_x[i], temp_list_y[i]]
                                temp_list_x_y.append(small_list)
                            list_BET.append(izotherm.SampleFeatures.Total_surface_area)         
                            list_X_Y.append(temp_list_x_y)
                            list_info.append([izotherm.FileName,izotherm.SampleName,izotherm.FigureNumber])
                            
    return list_X_Y, list_BET, list_info


oryginal_isotherms, bet, info = data_from_json()
print(f"oryginal_isotherms: {len(oryginal_isotherms)}")
print(f"oryginal_bet: {len(bet)}")
print(f"oryginal_info: {len(info)} \n")



    
def sieve_data(isotherms, info): # this function only change y value(frommm) in isotherms so dont need BET values also
    izotherms_numbers_to_change_y_value = [3,10,12,18,24,27,41,43,44,51,53,69,77,79,85,96,97,105,115] # here type numbers of Data[number]
    constant_to_change_y_value = 22.414
    new_list_of_isotherms=[]
    for index, isotherm in enumerate(isotherms):
        check = False
        for no in izotherms_numbers_to_change_y_value:
            if(info[index][0] == f"Data{no}"):
                check = True
        if(check):
            temp_isotherm = []
            
            if(info[index][0] == f"Data41"):
                constant_to_change_y_value = 1000
            else:     constant_to_change_y_value = 22.414
                
            for xy in isotherm:
                temp_xy = []
                temp_xy = [xy[0],xy[1]* constant_to_change_y_value]
                temp_isotherm.append(temp_xy)
            new_list_of_isotherms.append(temp_isotherm)

        else:
            new_list_of_isotherms.append(isotherm)
    return new_list_of_isotherms


def fit_shape_of_izotherms(oryginal_izotherms, new_size): #sorting, add or remove points between

    def sort_isotherms(oryginal_izotherms): #sort isotherm from (0,0) to (1,...)
        list_to_be_change = copy.deepcopy(oryginal_izotherms)
        izotherms_x_y_normalized = []
        for x_y in list_to_be_change:
            temp=[]
            temp = sorted(x_y, key=itemgetter(0)) # at index 0 from each element in x_y, so sorting by x value from 0 to 1
            izotherms_x_y_normalized.append(temp)
        return izotherms_x_y_normalized

    def add_new_points_in_first_iteration(oryginal_izotherm):
        izotherm_lenght = len(oryginal_izotherm)
        new_izoterm = []
        for p in range(izotherm_lenght):
            if(izotherm_lenght > p+1):
                new_izoterm.append(oryginal_izotherm[p])
                new_x = (oryginal_izotherm[p][0] + oryginal_izotherm[p+1][0])/2
                new_y = (oryginal_izotherm[p][1] + oryginal_izotherm[p+1][1])/2
                new_izoterm.append([new_x,new_y])
            else: new_izoterm.append(oryginal_izotherm[p])
        return new_izoterm            
        
    def add_new_points_in_specific_number(oryginal_izotherm, number_of_points):
        izotherm_lenght = len(oryginal_izotherm)
        new_izoterm = []
        for p in range(izotherm_lenght):
            if(number_of_points > p):
                new_izoterm.append(oryginal_izotherm[p])
                new_x = (oryginal_izotherm[p][0] + oryginal_izotherm[p+1][0])/2
                new_y = (oryginal_izotherm[p][1] + oryginal_izotherm[p+1][1])/2
                new_izoterm.append([new_x,new_y])
            else: new_izoterm.append(oryginal_izotherm[p])
        return new_izoterm
        
    def make_calculation(izotherm):
        return (2*izotherm)-1
        

    list_to_be_change = copy.deepcopy(oryginal_izotherms)
    list_to_be_change = sort_isotherms(list_to_be_change)
    #---------
    # result = [print(xy[0]) for xy in list_to_be_change ]
    # result = [print(xy[0]) for xy in list_to_be_change ]
    
    # result = [print(xy[0]) for xy in list_to_be_change if np.any(xy[0][0] == 0)]

    result = [xy.insert(0, [0,0]) for xy in list_to_be_change if not np.any(xy[0] == [0,0])]
    # lt = [print(xy[0]) for xy in list_to_be_change]
    # print(list_to_be_change)
    # print(len(result))
    #---------
    izotherms_x_y_normalized = []
    
    for x_y in list_to_be_change:
        izotherm_to_be_change = x_y
        old_size = len(izotherm_to_be_change)
        # print("old_size",old_size, end="")
        # print(f"old_size {old_size}")
        
        if(old_size < new_size):
            no_of_whole_iteration = 0
            temp_size = old_size
            number_of_additional_points = 0
            while(temp_size < new_size):
                if(new_size - temp_size+1 > old_size and make_calculation(temp_size) < new_size):
                    temp_size = make_calculation(temp_size)
                    no_of_whole_iteration  += 1
                else:
                    number_of_additional_points = new_size - temp_size                
                    temp_size = temp_size + (new_size - temp_size)

            new_izotherm_with_whole_iterations = izotherm_to_be_change
            for i in range(no_of_whole_iteration):
                new_izotherm_with_whole_iterations = add_new_points_in_first_iteration(new_izotherm_with_whole_iterations)
                # print(f"iteration[{i+1}] -> {len(new_izotherm_with_whole_iterations)}")
            final_izotherm =  add_new_points_in_specific_number(new_izotherm_with_whole_iterations, number_of_additional_points)
            
        elif(old_size > new_size):
            
            # regular_points = points[::2]
            # Liczba punktów, które chcemy wybrać
            num_points = new_size  # na przykład wybieramy 5 punktów z 10
            indices = np.linspace(0, len(x_y) - 1, num_points, dtype=int)# Obliczanie indeksów punktów do wybrania
            final_izotherm = [x_y[i] for i in indices]# Wybieranie punktów z oryginalnej listy na podstawie indeksów
            
        elif(old_size == new_size):    
            final_izotherm = x_y
            # print(f"temp_size {temp_size}")
            # print(f"no_of_whole_iteration {no_of_whole_iteration} ")
            # print(f"number_of_additional_points: {number_of_additional_points}") 
        # print("new_size",len(final_izotherm))
        izotherms_x_y_normalized.append(final_izotherm)

    return izotherms_x_y_normalized

# def add_zero_point(array):
#     print(len(array))
#     # [print(xy[0]) for xy in array if xy[0] != [0,0]]
#     result = [print(xy[0]) for xy in array ]
#     # result = [print(xy[0]) for xy in array if not np.any(xy[0] == 0)]
     
#     # result = [xy for xy in array if not np.array_equal(xy[0], [0, 0])]
#     # print(len(result))


def get_data(new_size, flat=True):
    isotherms_copy = copy.deepcopy(oryginal_isotherms)
    izotherm_x_y = sieve_data(isotherms_copy,info) # delete corupted data

    # #------------
    # add_zero_point(izotherm_x_y)
    # #------------
    
    izotherm_x_y_in_new_size = fit_shape_of_izotherms(izotherm_x_y, new_size) # make data with even points number 
    X_np_array = np.array(izotherm_x_y_in_new_size)
    y = np.array(bet)
    y = y.astype(float)
   
    if(flat):
        X = np.array([xy.flatten() for xy in X_np_array])
        print(f"flat=True, returned one dimension data")
    else:
        X = X_np_array
        print(f"flat=False, returned two dimension data")

    print("X",X.shape)
    print("y",y.shape)
    print("info",len(info))
    print("oryginal_isotherms",len(oryginal_isotherms))
    print(f"return: X, y, info, oryginal_isotherms")


    return  X, y, info, oryginal_isotherms




# elo = get_data_for_training_standarize(40)

# X1, y1, info1, oryginal_isotherms1=get_data(40,False)




oryginal_isotherms: 861
oryginal_bet: 861
oryginal_info: 861 

